In [36]:
import numpy as np
import pandas as pd
import os.path
from pandas import Series, DataFrame
import geopandas as gpd
from shapely.geometry import Polygon

In [3]:
data_files = ['traffic_density_202101.csv', 'traffic_density_202102.csv', 'traffic_density_202103.csv', 
              'traffic_density_202104.csv', 'traffic_density_202105.csv', 'traffic_density_202106.csv',
              'traffic_density_202107.csv', 'traffic_density_202108.csv', 'traffic_density_202109.csv',
              'traffic_density_202110.csv', 'traffic_density_202111.csv', 'traffic_density_202112.csv']
traffic_df = pd.concat((pd.read_csv(filename) for filename in data_files))
traffic_df.describe()

,LATITUDE,LONGITUDE,MAXIMUM_SPEED,MINIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
count,1.974586e+07,1.974586e+07,1.974586e+07,1.974586e+07,1.974586e+07,1.974586e+07
mean,2.888215e+01,4.106392e+01,9.775048e+01,2.531070e+01,5.626615e+01,7.190516e+01
std,3.551876e-01,1.002462e-01,3.515620e+01,2.464502e+01,2.478793e+01,9.411314e+01
min,2.796570e+01,4.074554e+01,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,2.869080e+01,4.099823e+01,7.000000e+01,6.000000e+00,3.400000e+01,1.300000e+01
50%,2.893250e+01,4.105865e+01,9.600000e+01,1.300000e+01,5.600000e+01,3.800000e+01
75%,2.914124e+01,4.113007e+01,1.240000e+02,4.600000e+01,7.900000e+01,9.200000e+01
max,2.988831e+01,4.164642e+01,2.550000e+02,2.430000e+02,2.430000e+02,1.351000e+03


In [4]:
traffic_df['DATE_TIME'] = pd.to_datetime(traffic_df['DATE_TIME'])

In [5]:
traffic_df['LATITUDE'] = traffic_df['LATITUDE'].round(3)
traffic_df['LONGITUDE'] = traffic_df['LONGITUDE'].round(3)

In [8]:
agg_traffic_df = traffic_df.groupby([pd.Grouper(key='DATE_TIME', axis=0, freq='3h'), 'LATITUDE', 'LONGITUDE']).agg({
    'MAXIMUM_SPEED': 'max',
    'MINIMUM_SPEED': 'min',
    'AVERAGE_SPEED': 'mean',
    'NUMBER_OF_VEHICLES': 'sum'
}).reset_index()

agg_traffic_df

,DATE_TIME,LATITUDE,LONGITUDE,MAXIMUM_SPEED,MINIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
0,2021-01-01 00:00:00,27.966,40.982,127,59,90.000000,16
1,2021-01-01 00:00:00,27.966,40.987,127,31,82.333333,16
2,2021-01-01 00:00:00,27.966,41.097,105,44,73.333333,8
3,2021-01-01 00:00:00,27.966,41.212,120,73,87.000000,22
4,2021-01-01 00:00:00,27.966,41.333,80,15,54.000000,8
...,...,...,...,...,...,...,...
6801450,2021-12-31 21:00:00,29.614,41.152,102,76,84.500000,2
6801451,2021-12-31 21:00:00,29.614,41.158,88,19,48.666667,4
6801452,2021-12-31 21:00:00,29.625,41.152,91,56,73.000000,4
6801453,2021-12-31 21:00:00,29.625,41.158,80,45,70.333333,3


In [9]:
def min_max_scaling(column):
    min_value = column.min()
    max_value = column.max()
    return (column - min_value) / (max_value - min_value)

agg_traffic_df['Normalized_MAXIMUM_SPEED'] = min_max_scaling(agg_traffic_df['MAXIMUM_SPEED'])
agg_traffic_df['Normalized_MINIMUM_SPEED'] = min_max_scaling(agg_traffic_df['MINIMUM_SPEED'])
agg_traffic_df['Normalized_AVERAGE_SPEED'] = min_max_scaling(agg_traffic_df['AVERAGE_SPEED'])
agg_traffic_df['Normalized_NUMBER_OF_VEHICLES'] = min_max_scaling(agg_traffic_df['NUMBER_OF_VEHICLES'])


In [11]:
w1 = 0.2  # MAXIMUM_SPEED
w2 = 0.4  # MINIMUM_SPEED
w3 = 0.3  # AVERAGE_SPEED
w4 = 0.5  # NUMBER_OF_VEHICLES

agg_traffic_df['Traffic_Density'] = w1 * agg_traffic_df['Normalized_MAXIMUM_SPEED'] + \
                                      w2 * agg_traffic_df['Normalized_MINIMUM_SPEED'] + \
                                      w3 * agg_traffic_df['Normalized_AVERAGE_SPEED'] + \
                                      w4 * agg_traffic_df['Normalized_NUMBER_OF_VEHICLES']

print(agg_traffic_df[['Traffic_Density']])

         Traffic_Density
0               0.308780
1               0.253185
2               0.244975
3               0.323441
4               0.153587
...                  ...
6801450         0.308284
6801451         0.159294
6801452         0.252727
6801453         0.222512
6801454         0.196632

[6801455 rows x 1 columns]


In [51]:
location_df = pd.read_excel('location.xlsx')
location_df['Location'] = location_df['Location'].str.replace("'", '')

location_df[['lat1', 'lon1', 'lat2', 'lon2', 'lat3', 'lon3']] = location_df['Location'].str.split(',', expand=True)

location_df.columns = ['name','Location', 'lat1', 'lon1', 'lat2', 'lon2', 'lat3', 'lon3']

location_df = location_df.drop('Location', axis=1)

location_df['lat1'] = location_df['lat1'].astype(float)
location_df['lon1'] = location_df['lon1'].astype(float)
location_df['lat2'] = location_df['lat2'].astype(float)
location_df['lon2'] = location_df['lon2'].astype(float)
location_df['lat3'] = location_df['lat3'].astype(float)
location_df['lon3'] = location_df['lon3'].astype(float)

In [61]:
location_df

,name,lat1,lon1,lat2,lon2,lat3,lon3
0,ADALAR,40.876377,29.095444,40.914627,29.148194,40.837376,29.039114
1,ARNAVUTKÖY,41.200000,28.733333,41.208395,28.749340,41.191604,28.717326
2,ATAŞEHİR,40.983333,29.116667,41.009525,29.212240,40.959029,29.072244
3,AVCILAR,41.015348,28.731462,41.106113,28.756692,40.969100,28.681979
4,BAĞCILAR,41.045556,28.840556,41.072214,28.872690,41.017565,28.807967
5,BAHÇELİEVLER,40.997500,28.850556,41.027850,28.884746,40.991660,28.805903
6,BAKIRKÖY,40.968155,28.822800,41.007142,28.896622,40.954508,28.773472
7,BAŞAKŞEHİR,41.077895,28.812551,41.149446,28.823924,41.057844,28.639130
8,BAYRAMPAŞA,41.048150,28.900455,41.077101,28.923980,41.025852,28.878448
9,BEŞİKTAŞ,41.068616,29.028536,41.106452,29.055098,41.036762,28.991826


In [58]:
geometry = [Polygon([(lon1, lat1), (lon2, lat2), (lon3, lat3)]) for lat1, lon1, lat2, lon2, lat3, lon3 in zip(location_df['lat1'], location_df['lon1'], location_df['lat2'], location_df['lon2'], location_df['lat3'], location_df['lon3'])]
gdf = gpd.GeoDataFrame(location_df, geometry=geometry, crs="EPSG:4326")

# Convert DF1 to a GeoDataFrame
gdf1 = gpd.GeoDataFrame(agg_traffic_df, geometry=gpd.points_from_xy(agg_traffic_df['LONGITUDE'], agg_traffic_df['LATITUDE']), crs="EPSG:4326")

# Perform a spatial join
result = gpd.sjoin(gdf1, gdf, how='left', op='within')


C:\Users\m1273747\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [60]:
result.describe()

,DATE_TIME,LATITUDE,LONGITUDE,MAXIMUM_SPEED,MINIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES,Normalized_MAXIMUM_SPEED,Normalized_MINIMUM_SPEED,Normalized_AVERAGE_SPEED,Normalized_NUMBER_OF_VEHICLES,Traffic_Density,index_right,lat1,lon1,lat2,lon2,lat3,lon3
count,6801455,6.801455e+06,6.801455e+06,6.801455e+06,6.801455e+06,6.801455e+06,6.801455e+06,6.801455e+06,6.801455e+06,6.801455e+06,6.801455e+06,6.801455e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2021-07-03 16:27:22.972216832,2.888234e+01,4.106509e+01,1.063008e+02,2.002925e+01,5.634445e+01,2.087537e+02,4.145700e-01,8.242489e-02,2.286961e-01,5.865435e-02,2.138200e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2021-01-01 00:00:00,2.796600e+01,4.074600e+01,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.646091e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2021-04-01 15:00:00,2.869100e+01,4.100400e+01,7.600000e+01,3.000000e+00,3.433333e+01,3.900000e+01,2.952756e-01,1.234568e-02,1.377410e-01,1.072840e-02,1.339120e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2021-07-07 00:00:00,2.893200e+01,4.105900e+01,1.050000e+02,8.000000e+00,5.600000e+01,1.110000e+02,4.094488e-01,3.292181e-02,2.272727e-01,3.105590e-02,2.106340e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2021-10-03 21:00:00,2.914100e+01,4.113600e+01,1.350000e+02,3.600000e+01,7.866667e+01,2.700000e+02,5.275591e-01,1.481481e-01,3.209366e-01,7.594579e-02,2.855053e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2021-12-31 21:00:00,2.988800e+01,4.164600e+01,2.550000e+02,2.430000e+02,2.430000e+02,3.543000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,8.905512e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,3.571340e-01,1.005030e-01,3.679153e+01,2.192709e+01,2.432723e+01,2.731705e+02,1.448486e-01,9.023495e-02,1.005258e-01,7.712325e-02,8.904102e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
psych_df = pd.read_excel('psych.xlsx')

,İlçe,Psikolojik Rahatsızlık_Evet,Psikolojik Rahatsızlık_Hayır,Location
0,ADALAR,2,360,"'40.87637720', '29.09544400', '40.91462700', '..."
1,ARNAVUTKÖY,11,819,"'41.20000000', '28.73333300', '41.20839490', '..."
2,ATAŞEHİR,45,1373,"'40.98333330', '29.11666670', '41.00952500', '..."
3,AVCILAR,2,1403,"'41.01534790', '28.73146180', '41.10611300', '..."
4,BAĞCILAR,4,2047,"'41.04555560', '28.84055560', '41.07221400', '..."
5,BAHÇELİEVLER,8,1910,"'40.99750000', '28.85055560', '41.02785000', '..."
6,BAKIRKÖY,16,807,"'40.96815500', '28.82280000', '41.00714200', '..."
7,BAŞAKŞEHİR,7,1395,"'41.07789500', '28.81255100', '41.14944600', '..."
8,BAYRAMPAŞA,18,846,"'41.04815030', '28.90045530', '41.07710100', '..."
9,BEŞİKTAŞ,5,759,"'41.06861600', '29.02853550', '41.10645200', '..."
